In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Bin and consolidate rare values
# Application Type

app_counts = application_df["APPLICATION_TYPE"].value_counts()
application_to_replace = app_counts[app_counts < 500].index
application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(application_to_replace, "Other")

#Classification
class_counts = application_df["CLASSIFICATION"].value_counts()
classifications_to_replace = class_counts[class_counts < 100].index
application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(classifications_to_replace, "Other")

In [4]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df).astype(int)
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [5]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"]
X = application_df.drop("IS_SUCCESSFUL", axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [6]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Import batch normalization to help stablize training by normalizing inputs layer by layer.

from tensorflow.keras.layers import BatchNormalization

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_3 = tf.keras.models.Sequential()

# First hidden layer with batch normalization
nn_3.add(tf.keras.layers.Dense(128, activation='relu', input_dim=X_train_scaled.shape[1]))
nn_3.add(BatchNormalization())

# Second hidden layer and a dropout of 20% of neurons randomly during training
nn_3.add(tf.keras.layers.Dense(64, activation='relu'))
nn_3.add(BatchNormalization())
nn_3.add(tf.keras.layers.Dropout(0.2))

# Third layer
nn_3.add(tf.keras.layers.Dense(32, activation='relu'))

# Output layer
nn_3.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the model
nn_3.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
fit_model = nn_3.fit(X_train_scaled, y_train, epochs=100, validation_split=0.2)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.6777 - loss: 0.6255 - val_accuracy: 0.7359 - val_loss: 0.5518
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7166 - loss: 0.5718 - val_accuracy: 0.7347 - val_loss: 0.5510
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7195 - loss: 0.5671 - val_accuracy: 0.7326 - val_loss: 0.5482
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7220 - loss: 0.5629 - val_accuracy: 0.7361 - val_loss: 0.5465
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7218 - loss: 0.5607 - val_accuracy: 0.7304 - val_loss: 0.5473
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7253 - loss: 0.5588 - val_accuracy: 0.7331 - val_loss: 0.5468
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7237 - loss: 0.5585 - val_accuracy: 0.7345 - val_loss: 0.5463
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7250 - loss: 0.5579 - val_accu

In [12]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 2ms/step - accuracy: 0.7298 - loss: 0.5578
Loss: 0.5577707290649414, Accuracy: 0.7297959327697754


In [13]:
# Export our model to HDF5 file
nn_3.save("AlphabetSoupCharity_Optimization3.h5")